# Dummy Implementation

In [1]:
from pydantic import BaseModel

import aloegraph.V2.graph.v2_aloe_node as v2_aloe_node
from aloegraph.V2.v2_aloe_graph import V2AloeGraph

class DummyState(BaseModel):
    status:str = "default_status"

class DummyGraph[DummyState](V2AloeGraph[DummyState]):

    def __init__(self, initial_state:DummyState):
        self.state = initial_state

    def method_1(self):
        pass

    @v2_aloe_node.AloeNode(is_entry=True)
    @v2_aloe_node.AloeEdge(target="node_2")
    def node_1(self):
        pass
    
    @v2_aloe_node.AloeNode()
    @v2_aloe_node.AloeEdge(target="node_3")
    def node_2(self):
        pass
    
    @v2_aloe_node.AloeNode()
    @v2_aloe_node.AloeEdge(target="alt_edge")
    @v2_aloe_node.AloeEdge(target="END")
    def node_3(self):
        pass

d = DummyGraph[DummyState](DummyState())

@d.AloeNode()
@d.AloeEdge("END")
def alt_edge(state:DummyState) -> DummyState:
    return state

print(d.generate_mermaid_chart())
d.render_mermaid_chart()

graph
__START__([__START__]) --> node_1[node_1]
node_2[node_2]
node_3[node_3]
alt_edge[alt_edge]
node_1 --> node_2
node_2 --> node_3
node_3 --> END
node_3 --> alt_edge
alt_edge --> END


# Dummy Initialization

In [2]:
# Chat State
import aloegraph.V2.agent.v2_chat_model as v2_chat_model
chat_state = v2_chat_model.ChatAgentState()

# Intake Agent
import aloegraph.V2.agent.v2_intake_agent as v2_intake_agent
intake_agent_state = chat_state.intake_agent_state
intake_agent = v2_intake_agent.IntakeAgent(intake_agent_state, {})

# Router Agent
import aloegraph.V2.agent.v2_router_agent as v2_router_agent
router_agent_state = chat_state.router_agent_state
router_agent = v2_router_agent.RouterAgent[v2_chat_model.ChatAgentState](router_agent_state, {})

# Chat Agent
import aloegraph.V2.agent.v2_chat_agent as v2_chat_agent
chat_agent = v2_chat_agent.ChatAgent(chat_state, intake_agent, router_agent, {})

# Refinement Agent
import aloegraph.V2.agent.v2_refinement_agent as v2_refinement_agent
import aloegraph.V2.agent.v2_refinement_model as v2_refinement_model
refinement_agent = v2_refinement_agent.RefinementAgent(v2_refinement_model.RefinementAgentState(), {},{},{},{},{})

# Invoke and Interrupt Test

In [3]:
import aloegraph.V2.v2_base_model as v2_base_model
import aloegraph.V2.agent.example.v2_echo_agent as v2_echo_agent
import aloegraph.V2.agent.v2_chat_model as v2_chat_model

class ChatNotifier(v2_base_model.ChatNotifier):

    def set_agent(self, agent):
        return super().set_agent(agent)
    
    def add_reply(self, msg: v2_base_model.ChatMessage):
        print(msg)

    def add_log(self, text:str) -> None:
        print(f"   LOG > {text}")

    def set_status(self, text: str) -> None:
        print(f"STATUS > {text}")

    def clear_log(self) -> None:
        pass

    def clear_msgs(self) -> None:
        pass        

echo_agent = v2_echo_agent.EchoAgent(v2_base_model.V2AloeConfig(), ChatNotifier())
echo_agent.compile()

echo_agent.state.user_message = "Hello?"
echo_agent.state = echo_agent.invoke(echo_agent.state)
echo_agent.state.user_message = "Hello during interrupt!"
echo_agent.state = echo_agent.invoke(echo_agent.state)



Graph compiled successfully.
> Hello?
Hello from Echo1
Hello from Echo2
Hello from echo3_interrupt
> Hello during interrupt!
Hello from echo4_resume


# Response Schemas

In [4]:
import aloegraph.V2.agent.v2_intake_response as v2_intake_response
print(v2_intake_response.IntakeAgentResponse.SYSTEM_INSTRUCTION("",""))


You are the Intake Agent for .

**What is ?**

> 

**Your role is triage:**
- Read the user’s latest message.
- Decide whether to respond directly or hand off to the Router Agent.

**Respond directly if:**
- The user asks about  itself (what it is, how it works, it's capability).
- The user asks meta-questions about the chat system, onboarding, or your role.

**Hand off to the Router Agent if:**
- The user’s request involves performing tasks, generating content, or using application-specific Route Nodes.
- The request requires knowledge of prior conversation state beyond the last few turns.

**Guidelines:**
- Keep answers concise and explanatory when responding directly.
- Always preserve the user’s intent: either answer their system-level question or forward their request to the Router.

**Output Description**
- Always return a JSON object with exactly this schema:
  {
    "description": "Response schema for the Intake Agent in AloeGraph V2.\n\nThe Intake Agent is responsible for tri

# Chart Renderings

## Chat Agent

In [5]:
chat_agent.mermaid_chart_png("img/chat_agent_chart.png")
chat_agent.render_mermaid_chart()

## Intake Agent

In [6]:
intake_agent.mermaid_chart_png("img/intake_agent_chart.png")
intake_agent.render_mermaid_chart()

## Router Agent

In [7]:
router_agent.mermaid_chart_png("img/router_agent_chart.png")
router_agent.render_mermaid_chart()

## Refinement Agent

In [8]:
refinement_agent.mermaid_chart_png("img/refinement_agent_chart.png")
refinement_agent.render_mermaid_chart()